In [1]:
import pandas as pd

In [2]:
cosine_sim = pd.read_csv('cosine_sim.csv')

In [3]:
cosine_sim_map = pd.read_csv('cosine_sim_map.csv', header=None)
cosine_sim_map = cosine_sim_map.set_index(0)
cosine_sim_map = cosine_sim_map[1]

In [4]:
from surprise import SVD, Reader, Dataset

reader = Reader()
ratings = pd.read_csv('ratings_small.csv')
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)
svd = SVD()
train_set = data.build_full_trainset()
svd.fit(train_set)

In [5]:
id_map = pd.read_csv('movie_ids.csv')
id_to_title = id_map.set_index('id')
title_to_id = id_map.set_index('title')

In [6]:
smd = pd.read_csv('metadata_small.csv')

In [7]:
def hybrid(user_id, title):
    idx = cosine_sim_map[title]    
    sim_scores = list(enumerate(cosine_sim[str(int(idx))]))    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)    
    sim_scores = sim_scores[1:26]    
    movie_indices = [i[0] for i in sim_scores]
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]    
    movies['est'] = movies['id'].apply(lambda x: svd.predict(user_id, id_to_title.loc[x]['movieId']).est)    
    movies = movies.sort_values('est', ascending=False)    
    return movies.head(10)

In [8]:
hybrid(1, 'Inception')

,title,vote_count,vote_average,year,id,est
4173,Minority Report,2663.0,7.1,2002,180,3.151891
7502,The Book of Eli,2207.0,6.6,2010,20504,2.878143
7208,Replicant,93.0,5.0,2001,10596,2.798160
6640,Déjà Vu,1519.0,6.6,2006,7551,2.771986
5580,The Three Lives of Thomasina,12.0,6.8,1963,15081,2.768020
7948,Stake Land,290.0,6.2,2010,52015,2.745156
8207,Looper,4777.0,6.6,2012,59967,2.742064
7901,Super 8,2496.0,6.6,2011,37686,2.737897
7828,I Am Number Four,1606.0,5.9,2011,46529,2.663714
5924,Fortress,171.0,5.7,1992,12088,2.649633


In [9]:
hybrid(2, 'Inception')

,title,vote_count,vote_average,year,id,est
7502,The Book of Eli,2207.0,6.6,2010,20504,3.794988
7208,Replicant,93.0,5.0,2001,10596,3.577295
6640,Déjà Vu,1519.0,6.6,2006,7551,3.540964
5296,Zardoz,106.0,5.8,1974,4923,3.538541
4173,Minority Report,2663.0,7.1,2002,180,3.521445
7901,Super 8,2496.0,6.6,2011,37686,3.504900
5580,The Three Lives of Thomasina,12.0,6.8,1963,15081,3.494643
7828,I Am Number Four,1606.0,5.9,2011,46529,3.486698
4596,The Core,531.0,5.5,2003,9341,3.457770
7948,Stake Land,290.0,6.2,2010,52015,3.449517
